# Load Input Data in Parallel with Dask and UXarray 

## Overview

This usage example showcases how to parallel load unstructured input data with the use of Dask and UXarray to minimize memory. 

## Imports
This notebook requires the following packages to be installed in the notebook environment. 
```
mamba install -c conda-forge uxarray 
```

In [1]:
import uxarray as ux
from dask.distributed import Client, LocalCluster
import xarray as xr
import warnings

warnings.filterwarnings("ignore")

## Reading Data with Chunking

### Data

Data loaded in this notebook is the simulated output from the Department of Energy (DOE) Energy Exascale Earth System Model (E3SM) version 2. The case is set up as an atmosphere-only (AMIP) simulation with present-day control forcing (F2010) at a 1-degree horizontal resolution (ne30pg2), where sea surface temperatures and sea ice set as default as in the E3SMv2 model. The case is run for 6 years.

Chunking can be done like Xarray, where simply add the `chunks` argument when calling `open_dataset` for reading a single file; or `open_mfdataset` for reading multiple files at once.

In [2]:
# Load a single file with chunking data to every 4 hybrid level at midpoints (by 4 lev)

data_file_monthonly = "/glade/campaign/cisl/vast/uxarray/data/e3sm_keeling/ENSO_ctl_1std/unstructured/20231220.F2010.ENSO_ctl.lagreg.ne30pg2_EC30to60E2r2.keeling.eam.h0.0006-12.nc"
grid_file = (
    "/glade/campaign/cisl/vast/uxarray/data/e3sm_keeling/E3SM_grid/ne30pg2_grd.nc"
)
uxds_e3sm_mon = ux.open_dataset(grid_file, data_file_monthonly, chunks={"lev": 4})

In [3]:
# Each chunk now contains 4 vertical levels
uxds_e3sm_mon.Q

<xarray.UxDataArray 'Q' (time: 1, lev: 72, n_face: 21600)>
dask.array<open_dataset-Q, shape=(1, 72, 21600), dtype=float32, chunksize=(1, 4, 21600), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 0.1238 0.1828 0.2699 0.3986 ... 986.2 993.8 998.5
  * time     (time) object 0007-01-01 00:00:00
Dimensions without coordinates: n_face
Attributes:
    mdims:         1
    units:         kg/kg
    long_name:     Specific humidity
    cell_methods:  time: mean

In [4]:
# Load multiple files with the same chunking
data_files = "/glade/campaign/cisl/vast/uxarray/data/e3sm_keeling/ENSO_ctl_1std/unstructured/*.nc"
uxds_e3sm_multi = ux.open_mfdataset(grid_file, data_files, chunks={"lev": 4})

In [5]:
uxds_e3sm_multi.Q

<xarray.UxDataArray 'Q' (time: 72, lev: 72, n_face: 21600)>
dask.array<concatenate, shape=(72, 72, 21600), dtype=float32, chunksize=(1, 4, 21600), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 0.1238 0.1828 0.2699 0.3986 ... 986.2 993.8 998.5
  * time     (time) object 0001-02-01 00:00:00 ... 0007-01-01 00:00:00
Dimensions without coordinates: n_face
Attributes:
    mdims:         1
    units:         kg/kg
    mixing_ratio:  wet
    long_name:     Specific humidity
    cell_methods:  time: mean

## Loading Data with `parallel` argument

Similar to Xarray, UXarray also supports loading data in parallel. Performance may not be significant due to the chosen dataset for this notebook; and Dask client configuration requires customization depending on the data. 

In [6]:
%%time
# Regular Load
uxds_e3sm_basic_load = ux.open_mfdataset(grid_file, data_files, parallel=False)

CPU times: user 18.8 s, sys: 407 ms, total: 19.2 s
Wall time: 21.2 s


In [7]:
# from dask.distributed import Client
# from dask.distributed import LocalCluster

import dask.distributed

client = dask.distributed.Client(n_workers=4, threads_per_worker=2)
# cluster = LocalCluster()
# client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/rtam/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/rtam/proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 235.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35877,Workers: 4
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/rtam/proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 235.00 GiB
Comm: tcp://127.0.0.1:35755,Total threads: 2
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/rtam/proxy/44115/status,Memory: 58.75 GiB
Nanny: tcp://127.0.0.1:36417,


In [8]:
%%time
# Parallel load
uxds_e3sm_parallel_load = ux.open_mfdataset(grid_file, data_files, parallel=True)

CPU times: user 13.2 s, sys: 507 ms, total: 13.7 s
Wall time: 18.4 s


In [9]:
client.shutdown()

In [10]:
uxds_e3sm_parallel_load

<xarray.UxDataset>
Dimensions:              (time: 72, n_face: 21600, lev: 72, ilev: 73,
                          cosp_prs: 7, nbnd: 2, cosp_tau: 7, cosp_scol: 10,
                          cosp_ht: 40, cosp_sr: 15, cosp_sza: 5,
                          cosp_htmisr: 16, cosp_tau_modis: 7, cosp_reffice: 6,
                          cosp_reffliq: 6)
Coordinates: (12/13)
  * lev                  (lev) float64 0.1238 0.1828 0.2699 ... 993.8 998.5
  * ilev                 (ilev) float64 0.1 0.1477 0.218 ... 990.5 997.0 1e+03
  * cosp_prs             (cosp_prs) float64 9e+04 7.4e+04 ... 2.45e+04 9e+03
  * cosp_tau             (cosp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 100.0
  * cosp_scol            (cosp_scol) int32 1 2 3 4 5 6 7 8 9 10
  * cosp_ht              (cosp_ht) float64 1.896e+04 1.848e+04 ... 720.0 240.0
    ...                   ...
  * cosp_sza             (cosp_sza) float64 0.0 20.0 40.0 60.0 80.0
  * cosp_htmisr          (cosp_htmisr) float64 0.0 250.0 ... 1.6e+04 1.8e+04
  * cosp_tau_modis       (cosp_tau_modis) float64 0.15 0.8 2.45 ... 41.5 100.0
  * cosp_reffice         (cosp_reffice) float64 5e-06 1.5e-05 ... 5e-05 7.5e-05
  * cosp_reffliq         (cosp_reffliq) float64 4e-06 9e-06 ... 1.75e-05 2.5e-05
  * time                 (time) object 0001-02-01 00:00:00 ... 0007-01-01 00:...
Dimensions without coordinates: n_face, nbnd
Data variables: (12/471)
    lat                  (time, n_face) float64 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    lon                  (time, n_face) float64 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    area                 (time, n_face) float64 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    hyam                 (time, lev) float64 dask.array<chunksize=(1, 72), meta=np.ndarray>
    hybm                 (time, lev) float64 dask.array<chunksize=(1, 72), meta=np.ndarray>
    P0                   (time) float64 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    ...                   ...
    soa_c1DDF            (time, n_face) float32 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    soa_c1SFWET          (time, n_face) float32 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    soa_c2DDF            (time, n_face) float32 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    soa_c2SFWET          (time, n_face) float32 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    soa_c3DDF            (time, n_face) float32 dask.array<chunksize=(1, 21600), meta=np.ndarray>
    soa_c3SFWET          (time, n_face) float32 dask.array<chunksize=(1, 21600), meta=np.ndarray>